## ECE 239 Final Project

In this project we explore various deep learning algorithms on the EEG dataset. 


In [10]:
# Imports and setup

import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import LSTM
from keras.layers import GRU

from utils.load_data import *
from models.cnn import CNN
from models.rnn import RNN

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# Loads data from the EEG dataset and removes all EOG data

person_train_valid, X_train_valid, y_train_valid, person_test, X_test, y_test = load_EEG_data()

print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


## Split into train and validation datasets

In [5]:
X_train, y_train, X_val, y_val = split_train_val(X_train_valid, y_train_valid, percent_validation=0.1)

print ('Training data shape: {}'.format(X_train.shape))
print ('Training target shape: {}'.format(y_train.shape))
print ('Validation data shape: {}'.format(X_val.shape))
print ('Validation target shape: {}'.format(y_val.shape))

Training data shape: (1904, 22, 1000)
Training target shape: (1904,)
Validation data shape: (211, 22, 1000)
Validation target shape: (211,)


## One hot encoding for all labels

In [6]:
print (y_val[33])
print (y_val.shape)
print (y_test.shape)

def convert_to_label(num):
    return np.array([1.0*(num==769),  1.0*(num==770), 1.0*(num==771), 1.0*(num==772)])

y_train_labels = np.array([convert_to_label(yi) for yi in y_train])
y_val_labels = np.array([convert_to_label(yi) for yi in y_val])
y_test_labels = np.array([convert_to_label(yi) for yi in y_test])

print (y_val_labels[33])
print (y_val_labels.shape)
print (y_test_labels.shape)


769
(211,)
(443,)
[1. 0. 0. 0.]
(211, 4)
(443, 4)


## Classifying data using a CNN

In [7]:
cnn = CNN(X_train, y_train_labels, X_val, y_val_labels, X_test, y_test_labels)
cnn.train(stride=2, optimizer='adam', epochs=5)
test_loss, test_accuracy = cnn.evaluate()
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

Instructions for updating:
Colocations handled automatically by placer.
Train on 1904 samples, validate on 211 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
1904/1904 [==============================] - 1s 413us/sample - loss: 1.4217 - acc: 0.2358 - val_loss: 1.3888 - val_acc: 0.2701
Epoch 2/5
1904/1904 [==============================] - 0s 242us/sample - loss: 1.3681 - acc: 0.3178 - val_loss: 1.4282 - val_acc: 0.2322
Epoch 3/5
1904/1904 [==============================] - 0s 237us/sample - loss: 1.3260 - acc: 0.3634 - val_loss: 1.4486 - val_acc: 0.2464
Epoch 4/5
1904/1904 [==============================] - 0s 236us/sample - loss: 1.2791 - acc: 0.3997 - val_loss: 1.4998 - val_acc: 0.2512
Epoch 5/5
443/443 [==============================] - 0s 124us/sample - loss: 1.4268 - acc: 0.3160
Test loss: 1.4267922473007615
Test accuracy: 0.31602708


## Classifying data using an LSTM Architecture

In [13]:
rnn = RNN(X_train, y_train_labels, X_val, y_val_labels, X_test, y_test_labels)
rnn.train(RNN_architecture=LSTM, activation="sigmoid", \
              optimizer='adam', epochs=5, batch_size=64, dropout=0.1)
test_loss, test_accuracy = rnn.evaluate()
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 1904 samples, validate on 211 samples
Epoch 1/5
1904/1904 [==============================] - 6s 3ms/step - loss: 0.6275 - acc: 0.6719 - val_loss: 0.5944 - val_acc: 0.7216
Epoch 2/5
1904/1904 [==============================] - 4s 2ms/step - loss: 0.4454 - acc: 0.8058 - val_loss: 0.5919 - val_acc: 0.7239
Epoch 3/5
1904/1904 [==============================] - 4s 2ms/step - loss: 0.3527 - acc: 0.8785 - val_loss: 0.5901 - val_acc: 0.7192
Epoch 4/5
1904/1904 [==============================] - 4s 2ms/step - loss: 0.2815 - acc: 0.9246 - val_loss: 0.6043 - val_acc: 0.7204
Epoch 5/5
1904/1904 [==============================] - 5s 3ms/step - loss: 0.2228 - acc: 0.9596 - val_loss: 0.5989 - val_acc: 0.7227
Test loss: 0.6121573761677366
Test accuracy: 0.7200902941264633


## Classifying Data using a GRU Architecture

In [14]:
rnn = RNN(X_train, y_train_labels, X_val, y_val_labels, X_test, y_test_labels)
rnn.train(RNN_architecture=GRU, activation="sigmoid", \
              optimizer='adam', epochs=5, batch_size=64, dropout=0.1)
test_loss, test_accuracy = rnn.evaluate()
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

Train on 1904 samples, validate on 211 samples
Epoch 1/5
1904/1904 [==============================] - 5s 2ms/step - loss: 0.6889 - acc: 0.6624 - val_loss: 0.6464 - val_acc: 0.6919
Epoch 2/5
1904/1904 [==============================] - 4s 2ms/step - loss: 0.4274 - acc: 0.8167 - val_loss: 0.6387 - val_acc: 0.7062
Epoch 3/5
1904/1904 [==============================] - 4s 2ms/step - loss: 0.3080 - acc: 0.8964 - val_loss: 0.6367 - val_acc: 0.7097
Epoch 4/5
1904/1904 [==============================] - 4s 2ms/step - loss: 0.2266 - acc: 0.9442 - val_loss: 0.6463 - val_acc: 0.7026
Epoch 5/5
1904/1904 [==============================] - 4s 2ms/step - loss: 0.1708 - acc: 0.9730 - val_loss: 0.6702 - val_acc: 0.7097
Test loss: 0.6635286232149628
Test accuracy: 0.6963882631964932
